# Data Science i skyen: "Azure ML SDK"-metoden

## Introduktion

I denne notebook vil vi lære, hvordan man bruger Azure ML SDK til at træne, implementere og forbruge en model via Azure ML.

Forudsætninger:
1. Du har oprettet en Azure ML-arbejdsområde.
2. Du har indlæst [Heart Failure dataset](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) i Azure ML.
3. Du har uploadet denne notebook til Azure ML Studio.

De næste trin er:

1. Opret en eksperiment i et eksisterende arbejdsområde.
2. Opret en Compute-klynge.
3. Indlæs datasættet.
4. Konfigurer AutoML ved hjælp af AutoMLConfig.
5. Kør AutoML-eksperimentet.
6. Udforsk resultaterne og find den bedste model.
7. Registrer den bedste model.
8. Implementer den bedste model.
9. Forbrug endpointet.

## Azure Machine Learning SDK-specifikke imports


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## Initialiser arbejdsområde
Initialiser et arbejdsområdeobjekt fra gemt konfiguration. Sørg for, at konfigurationsfilen er til stede i .\config.json


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Opret et Azure ML-eksperiment

Lad os oprette et eksperiment med navnet 'aml-experiment' i det arbejdsområde, vi lige har initialiseret.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## Opret en Compute-klynge
Du skal oprette et [compute-mål](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) til din AutoML-kørsel.


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## Data
Sørg for, at du har uploadet datasættet til Azure ML, og at nøglen har samme navn som datasættet.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## AutoML-konfiguration


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## AutoML Kør


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## Udrul den bedste model

Kør følgende kode for at udrulle den bedste model. Du kan se status for udrulningen i Azure ML-portalen. Dette trin kan tage et par minutter.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## Brug slutpunktet
Du kan tilføje input til følgende inputeksempel.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på at opnå nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi er ikke ansvarlige for eventuelle misforståelser eller fejltolkninger, der måtte opstå som følge af brugen af denne oversættelse.
